In [1]:
#the module 'sys' allows istalling module from inside Jupyter

import sys

!{sys.executable} -m pip install numpy
import numpy as np

!{sys.executable} -m pip install pandas
import pandas as pd

#Natrual Language ToolKit (NLTK)
!{sys.executable} -m pip install nltk
import nltk

!{sys.executable} -m pip install sklearn
from sklearn import metrics
#from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import  CountVectorizer #bag-of-words vectorizer 
from sklearn.decomposition import LatentDirichletAllocation #package for LDA

# Plotting tools

from pprint import pprint
!{sys.executable} -m pip install pyLDAvis #visualizing LDA
import pyLDAvis
import pyLDAvis.sklearn

import matplotlib.pyplot as plt
%matplotlib inline

#define text normalization function
%run ./Text_Normalization_Function.ipynb #defining text normalization function

#ignore warnings about future changes in functions as they take too much space
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

ERROR: Invalid requirement: '#visualizing'


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zhong\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zhong\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\zhong\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zhong\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  ['<', 'p', '>', 'The', 'circus', 'dog', 'in', 'a', 'plissé', 'skirt', 'jumped', 'over', 'Python', 'who', 'was', "n't", 'that', 'large', ',', 'just', '3', 'feet', 'long.', '<', '/p', '>']
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  <p>The circus dog in a plissé skirt jumped over Python who was not that large, just 3 feet long.</p>
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  [('<', 'a'), ('p', 'n'), ('>', 'v'), ('the', None), ('circus', 'n'), ('dog', 'n'), ('in', None), ('a', None), ('plissé', 'n'), ('skirt', 'n'), ('jumped', 'v'), ('over', None), ('python', 'n'), ('who', None), ('was', 'v'), ("n't", 'r'), ('that', None), ('large', 'a'), (',', None), ('just', 'r'), ('3', None), ('feet', 'n'), ('long.', 'a'), 

In [2]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        
def get_topic_words(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names())
    topic_words = []
    for topic_weights in lda_model.components_:
        top_word_locs = (-topic_weights).argsort()[:n_words]
        topic_words.append(keywords.take(top_word_locs).tolist())
    return topic_words

In [5]:
df=pd.read_csv("normalized_data.csv",index_col=0)

In [6]:
df.head()

,app_name,review_text,review_score,review_votes
app_id,,,,
251570,7 Days to Die,Early Access Review,1,0
226840,Age of Wonders III,Loved the old game and love this new one! Grap...,1,0
453960,Envy the Dead,"Could be good, though kind of isn't. plenty of...",1,0
2990,FlatOut 2,easily one of the best racing games I've ever ...,1,1
8930,Sid Meier's Civilization V,"It's an exhausting game to play, but once you ...",1,0


In [9]:
df_negative=df[df["review_score"]==-1]

In [12]:
df_negative

,app_name,review_text,review_score,review_votes
app_id,,,,
367450,Poly Bridge,Bought this game while it was on sale. Was tha...,-1,0
393380,Squad,Early Access Review,-1,0
301520,Robocraft,Early Access Review,-1,0
220240,Far Cry® 3,I don't know about the game. I could not launc...,-1,0
394310,Punch Club,The ending was so abrupt that when it rolled t...,-1,0
...,...,...,...,...
35140,Batman: Arkham Asylum GOTY Edition,I heard a lot of good things concerning this g...,-1,1
208580,STAR WARS™ Knights of the Old Republic™ II: Th...,Does not work on windows 7. Do not buy.,-1,1
265300,Lords Of The Fallen,Cannot progress after guardian boss. I enter t...,-1,0


In [13]:
reviews_negative=df_negative["review_text"]
reviews_negative.head()

app_id
367450    Bought this game while it was on sale. Was tha...
393380                                  Early Access Review
301520                                  Early Access Review
220240    I don't know about the game. I could not launc...
394310    The ending was so abrupt that when it rolled t...
Name: review_text, dtype: object

In [14]:
reviews = np.array(df['review_text'].astype(str))

In [15]:
normalized_corpus = normalize_corpus(reviews) 
normalized_corpus

['early access review',
 'love old game love new graphic battle vastly improve hard ai spell genrally good experience though slightly repetative awhile',
 'could good though kind plenty bug though idea good',
 'easily best racing game ever play amaze soundtrack nostalgia semi hardcore emo scene hour fun sale',
 'exhausting game play bnw expansion best strategy game hand edit play game long enough know quirk game still believe really good game future civ game love pitboss mode allow use shared steamcloud save give steam notification turn rather dedicated server run host game play could month potentially pitboss mode like even attractive future civ game focus around optimization lightweight ness pretty polygon perfect civ epic game spanding several month take second sign take turn leave game',
 'went gas station like price blew gas station',
 'real nito',
 'game know want action put anything else character cliche story little light main focus game gun play help game look amazing mood act

In [16]:
#define the bag-of-words vectorizer:
bow_vectorizer = CountVectorizer()

#vectorize the normalized data:
bow_corpus = bow_vectorizer.fit_transform(normalized_corpus)

In [17]:
lda_corpus = LatentDirichletAllocation(n_components=4, max_iter=200,
                                           doc_topic_prior = 0.9,
                                           topic_word_prior = 0.9).fit(bow_corpus)

In [18]:
no_top_words = 15
display_topics(lda_corpus, bow_vectorizer.get_feature_names(), no_top_words)

Topic 0:
game story like character really feel well gameplay time good play end great first much
Topic 1:
review early access best ever life ing kill zombie simulator man call gta guy cry
Topic 2:
game play fun good great like buy really love time hour recommend friend still worth
Topic 3:
game player like use time run even take new weapon thing need enemy start way


In [22]:
#prepare to display result in the Jupyter notebook
pyLDAvis.enable_notebook()

#run the visualization [mds is a function to use for visualizing the "distance" between topics]
pyLDAvis.sklearn.prepare(lda_corpus, bow_corpus, bow_vectorizer, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
3      269.562408 -124.803398       1        1  35.753437
0        1.782994 -295.079681       2        1  33.849765
2      220.811432 -343.832123       3        1  23.413920
1       50.533993  -76.050934       4        1   6.982878, topic_info=          Term          Freq         Total Category  logprob  loglift
108370    game  1.813019e+06  1.813019e+06  Default  30.0000  30.0000
217099  review  2.662120e+05  2.662120e+05  Default  29.0000  29.0000
82886    early  2.187770e+05  2.187770e+05  Default  28.0000  28.0000
10792   access  2.110800e+05  2.110800e+05  Default  27.0000  27.0000
194969    play  5.061820e+05  5.061820e+05  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
181181      oh  7.043313e+03  1.995733e+04   Topic4  -5.8268   1.6202
248097   super  5.917197e+03  2.134291e+04   Topic4  -6.0010   1.3789
112398    girl  4.489015e+03  8.524035e+03   Topic4  -6.2772   2.0205
73890      die  6.257690e+03  3.867332e+04   Topic4  -5.9450   0.8404
36084    black  4.600560e+03  1.098971e+04   Topic4  -6.2527   1.7909

[324 rows x 6 columns], token_table=        Topic      Freq       Term
term                              
10292       1  0.001812  absolutly
10292       2  0.001812  absolutly
10292       3  0.994147  absolutly
10292       4  0.000906  absolutly
10792       1  0.000005     access
...       ...       ...        ...
288665      4  0.997298         yo
290801      1  0.320640     zombie
290801      2  0.000073     zombie
290801      3  0.002590     zombie
290801      4  0.676696     zombie

[1088 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 2])

In [23]:
lda_topic_weights = lda_corpus.transform(bow_corpus)

In [24]:
doc_names = ["Doc_" + str(i) for i in range(len(normalized_corpus))]
topic_names = ["Topic_" + str(i) for i in range(4)]

#convert to dataframe
df_document_topic = pd.DataFrame(np.round(lda_topic_weights, 4), columns=topic_names, index=doc_names)
df_document_topic.head(5)

,Topic_0,Topic_1,Topic_2,Topic_3
Doc_0,0.1364,0.5909,0.1364,0.1364
Doc_1,0.1518,0.0404,0.4410,0.3668
Doc_2,0.3825,0.0714,0.3421,0.2040
Doc_3,0.3387,0.1160,0.4834,0.0618
Doc_4,0.0642,0.0159,0.3654,0.5545
